In [1]:
require 'onnxruntime'
require 'rmagick'
require 'torch-rb'
require 'numo/narray'

def preprocess_image(image)
  # Resize to 224x224 using RMagick
  image = image.resize_to_fill(224, 224)
  
  # Convert to RGB array and normalize
  rgb_data = image.export_pixels(0, 0, image.columns, image.rows, 'RGB')
  rgb_array = Numo::DFloat.cast(rgb_data).reshape(3, 224, 224) / 255.0
  
  # Apply CLIP normalization
  means = Numo::DFloat[0.48145466, 0.4578275, 0.40821073]
  stds = Numo::DFloat[0.26862954, 0.26130258, 0.27577711]
  
  3.times do |c|
    rgb_array[c, true, true] -= means[c]
    rgb_array[c, true, true] /= stds[c]
  end
  
  # Add batch dimension
  rgb_array.reshape(1, 3, 224, 224)
end

def simple_tokenize(text, context_length=77)
  # Simple tokenizer that creates zero array
  Numo::Int64.zeros(1, context_length)
end

def get_image_embeddings(model, image_paths, batch_size: 32, limit: nil)
 # Apply limit if specified
 image_paths = image_paths[0...limit] if limit
 
 puts "Processing #{image_paths.length} images..."
 
 embeddings_list = []
 
 (0...image_paths.size).step(batch_size) do |start_idx|
   batch_paths = image_paths[start_idx, batch_size]
   
   puts "Processing batch #{start_idx/batch_size + 1} of #{(image_paths.size.to_f/batch_size).ceil}"
   
   # Process batch of images
   batch_tensors = batch_paths.map do |path|
     image = Magick::Image.read(path).first
     preprocess_image(image)
   end
   
   # Combine batch
   batch_tensor = Numo::DFloat.zeros(batch_paths.size, 3, 224, 224)
   batch_tensors.each_with_index do |tensor, i|
     batch_tensor[i, true, true, true] = tensor[0, true, true, true]
   end
   
   # Create dummy text tokens
   dummy_texts = Numo::Int64.zeros(batch_paths.size, 77)
   
   # Run inference
   outputs = model.predict({
     "image_input" => batch_tensor,
     "text_input" => dummy_texts
   })
   
   # Get embeddings - ensure it's a Numo::NArray
   batch_embeddings = Numo::NArray.cast(outputs["image_features"])
   
   # Add debugging
   puts "Batch embeddings type: #{batch_embeddings.class}"
   puts "Batch embeddings shape: #{batch_embeddings.shape}"
   
   # Convert to torch tensor
   begin
     batch_embeddings_torch = Torch.tensor(batch_embeddings.to_a)
     norms = batch_embeddings_torch.norm(2, 1, keepdim: true)
     normalized_embeddings = batch_embeddings_torch / norms
     
     embeddings_list << normalized_embeddings
   rescue => e
     puts "Error in conversion: #{e.message}"
     puts "Shape before conversion: #{batch_embeddings.shape}"
     next
   end
 end
 
 # Combine all batches
 Torch.cat(embeddings_list, dim: 0)
end

# # Usage
# begin
#  # Load ONNX model
#  model = OnnxRuntime::Model.new('clip_image_text_encoder.onnx')
 
#  # Process directory of images
#  image_dir = 'house_data_png'
#  image_paths = Dir[File.join(image_dir, '*')]
#  puts "Found #{image_paths.length} images"
 
#  # Get embeddings for first 100 images
#  # embeddings = get_image_embeddings(model, image_paths, limit: 100)
#  embeddings = get_image_embeddings(model, image_paths)

#  puts "Final embeddings shape: #{embeddings.shape}"
 
# rescue => e
#  puts "Error: #{e.message}"
#  puts e.backtrace
# end

:get_image_embeddings

In [2]:
# embeddings

In [3]:
# Function to find nearest neighbors
def find_nearest_neighbors(embeddings, query_index, k=5)
  # Get the query embedding
  query_embedding = embeddings[query_index]
  
  # Calculate similarities with all embeddings
  query_embedding = query_embedding.reshape(1, -1)  # Make it 2D: [1, embed_dim]
  
  # Calculate cosine similarity
  similarities = Torch.matmul(embeddings, query_embedding.t())
  
  # Get top k (excluding the first which is the query itself)
  values, indices = similarities.flatten.topk(k + 1)
  
  # Convert to Ruby arrays and remove the self-similarity
  similarity_scores = values.to_a[1..-1]  # Skip first one (self)
  neighbor_indices = indices.to_a[1..-1]  # Skip first one (self)
  
  return neighbor_indices, similarity_scores
end

# Save embeddings to file
def save_embeddings(embeddings, filename)
  File.open(filename, 'wb') do |file|
    Marshal.dump(embeddings.to_a, file)
  end
end

# Load embeddings from file
def load_embeddings(filename)
  embeddings_array = Marshal.load(File.read(filename))
  Torch.tensor(embeddings_array)
end

# Encode text to the same vector space as images
def embed_text(model, text)
  # Create simple tokenizer (77 is CLIP's context length)
  dummy_tokens = Numo::Int64.zeros(1, 77)
  
  # Get text embeddings
  outputs = model.predict({
    "image_input" => Numo::DFloat.zeros(1, 3, 224, 224),  # dummy image
    "text_input" => dummy_tokens
  })
  
  # Get and normalize text embeddings
  text_embedding = Torch.tensor(outputs["text_features"].to_a)
  text_embedding /= text_embedding.norm(2, -1, keepdim: true)
  text_embedding
end

# Search by text query
def search_by_text(model, embeddings, image_paths, query, k=5)
  # Get text embedding
  text_embedding = embed_text(model, query)
  
  # Calculate similarities
  similarities = Torch.matmul(embeddings, text_embedding.t())
  
  # Get top k results
  values, indices = similarities.flatten.topk(k)
  
  # Convert to Ruby arrays
  similarity_scores = values.to_a
  result_indices = indices.to_a
  
  # Get matching image paths
  result_paths = result_indices.map { |idx| image_paths[idx] }
  
  puts "\nTop #{k} matches for query: '#{query}'"
  result_paths.zip(similarity_scores).each_with_index do |(path, score), i|
    puts "#{i+1}. #{File.basename(path)} (similarity: #{score.round(3)})"
  end
  
  return result_paths, similarity_scores
end

# Search by image index
def search_by_image(embeddings, image_paths, index, k=5)
  neighbor_indices, similarity_scores = find_nearest_neighbors(embeddings, index, k)
  
  result_paths = neighbor_indices.map { |idx| image_paths[idx] }
  
  puts "\nTop #{k} similar images to #{File.basename(image_paths[index])}:"
  result_paths.zip(similarity_scores).each_with_index do |(path, score), i|
    puts "#{i+1}. #{File.basename(path)} (similarity: #{score.round(3)})"
  end
  
  return result_paths, similarity_scores
end

# Usage example:
begin
  # Load model and embeddings
  model = OnnxRuntime::Model.new('clip_image_text_encoder.onnx')
  
  # Either generate embeddings:
  image_paths = Dir[File.join('house_data_png', '*')]
  # embeddings = get_image_embeddings(model, image_paths)
  # embeddings = get_image_embeddings(model, image_paths, limit: 100)

  
  # save_embeddings(embeddings, 'house_data_png.marshal')
  
  # Or load pre-saved embeddings:
  embeddings = load_embeddings('house_data_png.marshal')
  
  # Example searches
  search_by_text(model, embeddings, image_paths, "large kitchen island colonial")
  search_by_text(model, embeddings, image_paths, "white marble shower stall")
  
  # Search by image example
  search_by_image(embeddings, image_paths, 5)  # Changed from 505 to 5 since we limited to 100
  
rescue => e
  puts "Error: #{e.message}"
  puts e.backtrace
end


Top 5 matches for query: 'large kitchen island colonial'
1. din_1573.png (similarity: 0.262)
2. din_673.png (similarity: 0.259)
3. living_300.png (similarity: 0.257)
4. living_1333.png (similarity: 0.257)
5. din_300.png (similarity: 0.256)

Top 5 matches for query: 'white marble shower stall'
1. din_1573.png (similarity: 0.262)
2. din_673.png (similarity: 0.259)
3. living_300.png (similarity: 0.257)
4. living_1333.png (similarity: 0.257)
5. din_300.png (similarity: 0.256)

Top 5 similar images to bath_1006.png:
1. bed_1009.png (similarity: 0.993)
2. bed_1253.png (similarity: 0.993)
3. din_526.png (similarity: 0.99)
4. din_1260.png (similarity: 0.99)
5. kitchen_23.png (similarity: 0.989)


[["house_data_png/bed_1009.png", "house_data_png/bed_1253.png", "house_data_png/din_526.png", "house_data_png/din_1260.png", "house_data_png/kitchen_23.png"], [0.9927145838737488, 0.9927145838737488, 0.9897346496582031, 0.9895848631858826, 0.989266037940979]]

In [4]:
require 'rmagick'
require 'base64'

def display_image(image_path)
  # Read image using RMagick
  img = Magick::Image.read(image_path).first
  
  # Resize if too large (optional)
  if img.columns > 500 || img.rows > 500
    img = img.resize_to_fit(500, 500)
  end
  
  # Convert to blob and base64
  blob = img.to_blob { |b| b.format = 'JPEG' }
  base64_img = Base64.encode64(blob)
  
  # Display using IRuby
  IRuby.display IRuby.html "<img src='data:image/jpeg;base64,#{base64_img}'>"
end

# Modified search functions to display images
def search_by_text(model, embeddings, image_paths, query, k=5)
  text_embedding = embed_text(model, query)
  similarities = Torch.matmul(embeddings, text_embedding.t())
  values, indices = similarities.flatten.topk(k)
  
  similarity_scores = values.to_a
  result_indices = indices.to_a
  result_paths = result_indices.map { |idx| image_paths[idx] }
  
  puts "\nTop #{k} matches for query: '#{query}'"
  result_paths.zip(similarity_scores).each_with_index do |(path, score), i|
    puts "#{i+1}. #{File.basename(path)} (similarity: #{score.round(3)})"
    display_image(path)
  end
  
  return result_paths, similarity_scores
end

def search_by_image(embeddings, image_paths, index, k=5)
  puts "\nQuery image:"
  display_image(image_paths[index])
  
  neighbor_indices, similarity_scores = find_nearest_neighbors(embeddings, index, k)
  result_paths = neighbor_indices.map { |idx| image_paths[idx] }
  
  puts "\nTop #{k} similar images to #{File.basename(image_paths[index])}:"
  result_paths.zip(similarity_scores).each_with_index do |(path, score), i|
    puts "#{i+1}. #{File.basename(path)} (similarity: #{score.round(3)})"
    display_image(path)
  end
  
  return result_paths, similarity_scores
end

# Usage example:
begin
  # Load model and embeddings
  model = OnnxRuntime::Model.new('clip_image_text_encoder.onnx')
  
  # Generate or load embeddings
  image_paths = Dir[File.join('house_data_png', '*')]
  # embeddings = get_image_embeddings(model, image_paths, limit: 100)
  
  # Example searches with image display
  puts "Searching for: 'large kitchen island colonial'"
  search_by_text(model, embeddings, image_paths, "large kitchen island colonial")
  
  puts "\nSearching for: 'white marble shower stall'"
  search_by_text(model, embeddings, image_paths, "white marble shower stall")
  
  puts "\nSearching by image index 5:"
  search_by_image(embeddings, image_paths, 5)
  
rescue => e
  puts "Error: #{e.message}"
  puts e.backtrace
end

Searching for: 'large kitchen island colonial'

Top 5 matches for query: 'large kitchen island colonial'
1. din_1573.png (similarity: 0.262)


"<img src='data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAMCAgICAgMCAgIDAwMDBAYEBAQE\nBAgGBgUGCQgKCgkICQkKDA8MCgsOCwkJDRENDg8QEBEQCgwSExIQEw8QEBD/\n2wBDAQMDAwQDBAgEBAgQCwkLEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQ\nEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBD/wAARCAEAAQADAREAAhEBAxEB/8QA\nHQAAAgMAAwEBAAAAAAAAAAAAAAYEBQcCAwgBCf/EAEIQAAIBAwMDAwIFAwID\nBQcFAAECAwQFEQAGIQcSMRNBUSJhCBQVcYEjMpGhsRYX8DNCwdHhJCVDUmKC\nsjREcnOi/8QAHAEAAgMBAQEBAAAAAAAAAAAAAAIBAwQFBgcI/8QAOREAAgIB\nBAECBQMCAwYHAAAAAAECAxEEEiExBRMiBiMyQVEUM2FCcSSx8BUWgZGhwQcm\nNDVigtH/2gAMAwEAAhEDEQA/APy7gulVb5CkcodMkkEHBPv558586hoCdRXC\n1S1UVaymkq4ZFkWQElSwIIJH7jRgDtrNsNLVXMUEqBaXsnijcjMkMhBQg55I\nDLkYwM8kY1LYLkoJqeopnKTxOjDyGGNRknBfXYettizT8nsp5Ys/cTsSP8OP\n9NNJIlPgXMcZ1Apc0M0yberYgB6bVEJPznskx/H/AJjSy5YyZTA40woyhjS7\nEBB//W17Z+cIi4P+WOoY0eELXt++pFPozkDRnHJKWWPN/r5LJtWh2zJEYqgR\niR+cgl/qP7EBgCPkHWeKzPcXt7UJCI0jhVBLMcD5JOtLe1FC5Y63kPtXbkW3\nFbNZXAS1Kj/ukgAD/Bx/GqIrc8lsvloTCO5xHGMnhR99aVyUp4Re2q0vcrut\nLHH3Q07J6jHyAM5A+xOf8DVN8sLA9ay8ndvbcH6rWLRQPmlpCVTHuRkE/sOQ\nP3J99LRXhZHtlko7Zaqy71sdBQxGSWU4UD/x1ZKe0rjHcPtz3DR7IsNTs+yj\n1quoB/M1QKkBj29yFSCGGUUjGCrKrA8Y0kG5djSxEodmXm9LdKW10IlqI5nE\nZiX+7tJwQD7Hnj74/bRKCIjJmi7orKWn3PUxvNFSVF0JqldiXRI3wUEZJ+rj\ntJIwM54GMaonXseUWRnnsz3cmzb/AErLcZc1n5mUqZIxkZPK/wAEZ9gBgj4J\nuhZlckThnot6CgtfT+hS8XWRKm61CBoIY3BEYIyCT8+Mn2OlcvV4IS2oSbtd\nq2+Vj1tbJ3SN4A4Cj2A1bGKgV9ssLLtaauT9QuMoo7euC0r8Fh8KD51ErNvC\nJjHPZIuW5kSn/SdvQmClA7e48u/nn+cn29zjGojDPLGlLb0V9p27XXPMxHoU\nyf3zScAD7fJ/bVkpY6ESLGa/UVjRqOwKS2CrTvzknGSP8ft40vM+xlLAr6YQ\nNABoANAFldFWojguqLj8wCkvOf6ygdx855BVs4AyxA8aZkIrR50pIxy1EFTt\n2lq4e9K2izSsyeCoYyIxPyQXUDniP/DOILg6aXdFVGyrXU8VZH29rCRQSRnP\nn5z76TBORiCWG7WKz1H5eWnphdJ4KoISe1GERGABjIAPjzjnxqMNEwWSjG0X\nrFMtmroqtQO7t8MP3B9/40vqYGUDvuNgr7FtaJ66IIa2cyqOc9iqoyePlz8j\nU5yyHHArtwAceRn/AF1Y+hBp3RRi1WWy24kh2o0qpFJ8mb+oDjyPpIGfBwNI\nuWPjCFX4xphBj2JZortfo3q1JpKFTVVBxx2r4Hxy2B/Oq7XiJZWssibmuct4\nvFTWSSd6lyFI8EZPj+cnTVx9uWTaxm6cbchEFVvi8oBbrV/2YbxNPjIAz5A4\nJ+5GPB0lksvA1UMrcxUvl1nu1xnutQT3zsewf/Kntj/b/OrILCKrJb3gkbcs\n4vMpipJ4zWoe+OmY9rTgDJWMnhnz4U8ngKGJxqVLaiMZZatXrY9tpDBG8Vfc\nu9pmkXtcL3EBgfgqQB98kYwc1fuMt+hCylLJVTpTU0ZkkdgqKnJJJ4A/6/21\nbKW1CqO8eTVUGwrMsFA6y3utiDSSjBECkEEfY+Rjg+SdUxXqMZ+wTaSgrrxW\nLS0kbT1E75A8knnJJ+Pk6uk1BFcYuQ1SXKm2BEKO0MDelIMlSrHMTA5BU+xB\nwRgjkZ/emLc2PlQFelpbxua8CKlhepq6lyxCKAMk5J4wFGT9gNWvCXIiTm+D\nV7tXWrp/s+kttXdBcL2gLLCHDJC5Zg+RnIIQoQ5BBGQDkYFChvfBapen2V22\n7btLqJTz0j0ktPXxU+I2DEvLLg9ig85yc+B7YIAOQqzWH1oq6nZVp2uj1lVJ\nLc6ozYp4ET6VHdwZCDjJGMjOBnTOxy4FUcMVr/d7xe68U04DBMJDBCPoA9sA\necgg5986sjDHLEb3dFzb9rUFhiS5bpZjOwDQUMTfWx9i2PAzqXPfwiVHHZA3\nTfKitSGFKgRqO4tTRDCRjjtB+T5zn/HuWVbh2JuyVltsNfcl9YJ6UKjJlk4G\nPt8/xqJTXSJUclXpiA0AGgA0AWlqH5uOotWeahA8QzgGZASo+SSC6AeMuPjT\nkYKwgqcHUYJLqwF6uluVpDPiaAVMca+GkhPcWP2ERnP741IFKwKsQfIJGkAb\ndoVzfo13trQ+qqCKtGcAIQ3pZBIJ5My5xgkDyNEmCQswTTQO8sErxsgBypIP\nkD/x0RWSc4H24boNVsuz1tRURTVtM01JLC4X+ogMZQuCcnhRzjGRjIxg1Sy2\nMmRbRSbT3bV01vamloa2bOPROUwMkgAjzgEgZOcgDUPKGTTLLf8Atesvt9rq\nzb8lPV0lFM1vgSOTDtFAiqjhTg4IIAPyD8cxCaj2NKOejPam21tDL6NXSyxO\nP+66kHVqnCXRU4sdo45NqdO1kZXjrNxSF1zwRBGxVQQeRlhI2PfCnPgaq+uR\nojJVR/uKVot1RerjS2egiMlVVSrBGMeWJwCfgDPJ9gM60SlGMeSiFe6XA6dR\n7rFboqPpzaqkNQWgBZXTj1ZTy5I9yCSf5A5xnWepKx7i+1qpbEZ/UFpHL9uF\nAwo+AOANadq6Rlb5waX0xtdpsVrrOod7cAUf9OigZeWcgjvGeDzkA8+GGAcH\nWDVTc5+nA10w2rMhFvVwrL7dZrhOGaSd/pUc4GeFA9vjHknWuuKrjkom/Ulh\nDrFbafpnaFr7rFHJuGviBhhbk0aEYyR7Nggn4yAOc6ocndLjov2quO4SqSju\ne4LmtPTRSVNXVPgKPLEnkn4A858Y1fNxqjwUwza8scLjUUHTmge10Miz3ypQ\niomGCIgfYcePt7nk8azxjK1lspKpYQpbe29eN3XVaWkBLysWlmk4VRgkkn9g\nT99XTkqkUxg5vLHW4bisewaGSwbMKVVykHZVXDAIU+4U+Cc5wPA986SuMreW\nWTkquIivt/aV03TNPcKyrFNSIS9VW1DcDPJPPJJwePkjTyt2cQK1Fy5ZetvG\nl2bTzW7YVP6ZmiEctymUid2DBg0ZBBQggEH44wQTkhBz5mRKWzhEbp2N0X7c\nn5WnjqK2GUyTVQ8AAAsSCQQpJXA4wTgYPjROKfESYtrljjuOr2zb923o7YqL\nW9xeSQwzkl0iUuxAjOB3HsK8nn2IGMGp70/4LYxWBCuG2t5fqbQSxvUTVIWR\npSRlFKK47icdv0upIPAJI8g6tVkEv5KpQyzgtvsW3HY3WQXGsADJHHkxg85y\nfcgjB9tRJyn10CionVONwbjIJUUtGBhVz2oAPn5/640LbEh7pdCtq0QNABoA\nNAHbBNLTzR1FPI0csTB0dTgqwOQQfkabIFheaaL+lcaWPtgq19QAHIRvDp9s

2. din_673.png (similarity: 0.259)


"<img src='data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAMCAgICAgMCAgIDAwMDBAYEBAQE\nBAgGBgUGCQgKCgkICQkKDA8MCgsOCwkJDRENDg8QEBEQCgwSExIQEw8QEBD/\n2wBDAQMDAwQDBAgEBAgQCwkLEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQ\nEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBD/wAARCAEAAQADAREAAhEBAxEB/8QA\nHgAAAQQDAQEBAAAAAAAAAAAAAAQFBgcDCAkCAQr/xABKEAACAQMDAwIDBQYD\nBQYCCwABAgMEBREABiEHEjETQQhRYRQicYGRCSMyobHBFdHwFyRC4fEWJzNS\nYnKStBglJjdDY2R2gqKk/8QAGwEAAgMBAQEAAAAAAAAAAAAAAAQCAwUBBgf/\nxAAuEQACAgICAgICAgAFBQEAAAAAAQIDBBESIRMxBSIyQRRRI0JSYXEkMzRD\ngcH/2gAMAwEAAhEDEQA/AOqegA0AGgDQb9qWMbi+HR/YdQYR+r05/toA34Aw\nNAH3QAaAPh/tqE/QI4BfFtbXpLLtCpcHIjqlPyziI/11k4UEpSNDJf1R1M6Y\ns8vTXaUkZPY9joGQ59jBGR/LGnmRj+KJxQBzTrlmBBIAz9dBIVKXBGWbP46A\nMq9wI++fzJOgDIobg95/U6APYWQjPcefkToA+hJPd2H5nQB5JYHHe36nXQfs\nrD4o52T4cupJLHB21XqfzhYf3/nrtfshP0ci9pbTpb3X7UjqJVEdwuFyp5E9\nAOf3dJ3gkE/eySABxg8gk+Fcq2XGeieNVXY1yLWoOh+wnrVWs7XJTtAWCBFI\nAz47SSfOSST4z41jyzL4y6RpvEojvRkq+inSqKOnjkgnM1TPFGgWZY8R+lMz\nnCqBwyxAEjP3iM+2r45mSo+iEsXGbPi9GOj0KhXoZHJPl7jKCT9ArD5fnjVK\nzsuMvRNYmNowT9H+kx5hhli4/wCC5S/3Y6ms/L12iKxcaX7K76l7K2ft2v2t\nS2CrqiLrdUo6zuqxIUiLKMrx904J5OfbjjWng5Vl2/IJZVFdf4kil6PbOjaV\n3v8Ad6bvkdos1Mf3oix9NuUGcpjn3IPy1VkZlsZfVEacWmz2IJekG2CxEO+6\n1MA/xmNvbOOAB/r31xZ1sl2iUsOmD6Zii6LWqVi6dSO1QCe56dSMD2H3xk59\nh+eNTWdP+iDxI/6jJJ0GQIZY+olKwUgMWomwpKk9pw+Q3DDGMkqfbBNqzpf0\nVvCj/qNX9aQkGgA0AGgD9VGgA0AGgDRH9qLSmet+H6ZcZTqPSr9cHsJP4fdH\n8tQA3kmuVvpxmorqeID3eVR/U6lsBiuHU3pvaWZbt1A21RFRkiou0EZA+vcw\n1FyAjNf8Tfw42titw699O6dh5WTc9ED+hkzo5AMtb8Zfwo0aFpviI6fsB59K\n/U8pP5IxP8tcb2gRxo+LHqP056g7f27aNg7kiu1XbJ5TULHTzRBUKqAcyIoI\nJHsSePGksSlwcmxi6e4o6pdHKX/uj2QceduWz/5WPV76Lofiie0dPinUY9z/\nAF1wkKVgxz26APaxHABGD89TAyonIGdAHv0/roA8kefpoA8EY1x+gfsp/wCL\n6V4Phm6itGhZmsksYGfdiF/vrlfTIT9HOvol04637ibpuuxNj7Ar5Lnui+U1\nm/7R2+KV5K2O0mSqjqhJGe6nFOA0Sk4WYd3nBFzTl7Fm9a4my/8A9Hz496CX\n1Kb4ePhzkdCSrRbXtKknkcElSM5Py9z51Hxx/aOc59nxel37QW3xSQQfB50B\nnEiCKXs29ZlEqAhgpH2xQRlVOCByAfI0KqP9BtjRX7J/aBUAeV/gN6Gyh0Mb\nrBtqzv3KSCQQteSRkA45+ftxLxQX6OubREL9D8a1sci5/s5OkVRgjmn6aQ1Q\nPPjMNQ3z/wBDXPHBrbRxzl+mUL1O3X1bqr7tG1bt+ELZ2zLlRbkp6i10dr6e\nzUD3isVgFoJkZiaiJ2AUwryxJA86jBwb+p2Sm12y1rn1V+Kb7ZJW339m/wBO\npZ3CK71PRq5k4VAiDlscKqqMYwAAManpftEU5JdMb6j4i+uVm7Xun7P3pFRZ\nBUet0ouEHBBBAy44IZhgecnzzqLgpekSjOX7ZH6z4zd6WditZ8GHQakypHbN\n0/qU4II/4pRjk+f6YGjhD+g5yEy/HjXRKYpfhA+HsjH3u/Z0uMEAEYM3AOAC\nPHA1LhD+g5SNONWFYaADQAaAOjy9Zf2tl7kMUVH1Ti78Af8Ad3TU68/+pqRc\nDnzkap3MnpCiXpj+1u3zGZpbt1MgDgk//aeltGR/7FqY8H8gdG5nNIxQfB/+\n1CvTCK57t3vSqSO41nU7vU/UiOocn9DqXENlX9ffg0+J3pVLsy5dUt2Wi7y7\nsvSWG3ie9VFxenqpcEGQyxYVTjJKFjx4PA0aOltRfsdOrwhq7rvPqrsOhiiR\n6iaWkirarAALMcMkXHHgfgBqMvqcTRIdk/seqS+2iK6XD4hY6V3lkjMEO1CR\ngStGjBnqwSHK5H3fJIGcE6jCakHZZlm/Y0dK4O3/AB7rDumtx5+yUFJTfp3r\nJj+epqAJ6RJ4v2PvwyRqPtG7Ook59wbjRIDx7dtICP11zjo4vZz4+J74c9k9\nF9uWm87WqLxLUVNa9LUfbqtZRgIWHYFRcEkf20rRc5SaL5x+qOrXSGmx0k2U\neTnbltP/APlj1fraL4/iidU0H7j+H566dPQhwQSONAHvsUeNAH0AD8dAARka\nAMcnAx9dAGJtQB+yk/jGuU1p+GjfddTgF46OHtBLDk1MQ8qQR58ggj2I1L9E\nZs1w+DTdFHJ1P6E7JhoPTp6PeO7rpS1sciFJ++xPD6RQYK4GGDAdpB7QAQcF\nU/7KLV+zpd1P3VddlbHu+6bHZGvFfbqczQUASob7QwI+5/u8M0oyM8iNgDgn\nAyRbJplcI7EXTLqJFvXYNi3ZdIFoKy60UVXPRLFOpp2cA4CzRxylBniRo1DD\nDAYIOhWJnJx0ybowZe5TweRqfTIjZfqyGmozBM0imq7okZMkhu1myMc5AUkY\n5yOOdck/qcX5HND4yq5G+IboTf4VQU9w3/t+6RRpMJD9+aMyKTxyJklXHgBV\nBwCAEsdfdjNj+p1AjcOodSCGAI/A6d01IXPfB0dp7A+6kB8IBGCBjQB+VjQA\naADQAaAP1T5+uudgGfro7A+66c2aW/tIR6lb0FpSfuydSqFiMfLA/uf11XKR\nbA2n6k1K0mz6ud09T05KeQIf4ZWWZGETcH7rkBCcYw3JA5ELvw2QW3LSNOaL\n4hdoWm7UyVHUWaltkFdTOlvhojPFHbxNGBAGjhJBhiLshJLKyhQWLuUw4ZMo\nWaHv4tso7Rtbsv4hOh3UFoafZ3Vja1yqplJWijucS1Yx5zAxEq4+qjW/Ge0J\nSraej50/65bE6nb13jsfZ1RU1tRsiWmguFaiK1HLLMHPZDKrHuZDGyuCBgjj\nPJEd+zrg0cpP2iUap0p2zLHgF72q5U5BBp5DwfcZGQdKYjTky21/RHR3pFCD\n0l2VhcgbctuD8/8Ado9NS6RdDtImsCAR+MEE64SPpXjzoA8FATwcDQB5YYON\nAGMnGdAGGR8549/noATSPn2+fvqD6OspD4zrXdL38Mm+bVZbVW3KtnpqUR0t\nFTvPLIBWQM5CICSFQMxwDgAk8AkSj2VzObPTz4ht3dGN2bV3Lt+3WeS4bCrD\nVUdPcqWQhpa63SU9Qk4WSNzhSSoyCpAzkcaIxRVYzZwftVr1uyEWvd/TqeCn\nngMFTJZr0fvBlKsEgaIZBBzh5GOTjODjRKtshBj5sP45OiVLZ6Lb9XW36xLa

3. living_300.png (similarity: 0.257)


...

4. din_1260.png (similarity: 0.99)


"<img src='data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAMCAgICAgMCAgIDAwMDBAYEBAQE\nBAgGBgUGCQgKCgkICQkKDA8MCgsOCwkJDRENDg8QEBEQCgwSExIQEw8QEBD/\n2wBDAQMDAwQDBAgEBAgQCwkLEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQ\nEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBD/wAARCAEAAQADAREAAhEBAxEB/8QA\nHgAAAgICAwEBAAAAAAAAAAAAAAYFBwQIAQMJAgr/xABDEAACAQMEAQIEBAQC\nBwcEAwABAgMEBREABhIhBxMxFCJBUQhhcYEVMpGhI0IWJDNSscHRFyVigsLS\n8BgnNfFDcuH/xAAcAQEAAgMBAQEAAAAAAAAAAAAAAwQBAgUGBwj/xAAsEQAC\nAgICAgIBBAICAwEAAAAAAQIDBBESIQUxE0EiBhQyUSNhFXEzQoFi/9oADAMB\nAAIRAxEAPwD1T0AaANAGgDQBoA0AaANAGgOM50AYGhjZzoZDQBoA0AaANAGg\nDQBoA0Bxn8tDGg7P5aD0c6DYaDYaDQaDR+VfQyGgDQBoD9U+RoDjkNAHMfYn\n9tAHIfQH+mgDmPsf6aAOY/PQByH0B/poALgfQ/00Ach+egOPUTOOQz9vroA5\np99ABkTOOXf2+ug47OOQIz/f/wCe2g0c8x/8xoZ0wyP/AIdDBxyXP10Acl+x\n0ByHA9gf6aACyk9g6GNnHJfsevrjQdhyA+5/TQdhzX6Z0HYCQZ/lOhlJnPMN\n13oYlA45f+E/01nZrwZzkfmf202ODOQ3X11g24hn8z/TQcT8rOhkNAGgDQHq\nI3nLzQKqZR5Y3fgSMAP41UYABOP8+gM6Dzd5jJ78q7tP63moP/q0BIReafLp\nHzeUt1k/nd5//doDKTzR5cJH/wBz90kY+t2n/wDdoDNg8x+WG9/Jm6CMZ/8A\ny05/9WgM+Hy/5SckN5I3Kfp/+Un/APdobqJ1V3k3y5EDPB5N3UYj2R/Fp8r+\nX83t+egaOyj8w+VJGAk8jbmyMZBuk3/u0NBhofK/kvoy7+3Aw6OTcZj/AOrQ\nDFTeSN/SBXO9b2SR3/r0h/56AnLfv/ecp4vuy7kkAjNZJ3/fWWtGdEtWbk3h\nfbJVWj/TG80zVMLRLUQV0kUkRI6YOrBgQcHIIOtGyWrW+zTzyn5S/EDtIivT\ny1vX4zalYlLdaaO9VIFVAobjOUDgN6kOScAASxEAjPcbsaOpVVVNb0LXkTzZ\n50m29T7i275s3xGtIyzy/D7gqUEsDj36cAgAgg4+30Oo3cyX4KpdJHftX8RX\nmOqpYjL5i3hMcDK/x2pkIOB7kv8A8dbO2ZmGNU/ocaXzf5kd0J8n7qClhnle\nakkj8vmAGfb6/tqP5pm8sWr+ibtHmDy/W1QA8k7uq3AwtPBdKhmY/mFYnA67\nOP8AmLNe5srzhREmLt5Y82We1Nc6rft+jQTJCVN4nLgsCR8qFielOR1+up7E\n4IqquF74xFJ/PHmmrkPq+SN1KhOFH8UnhBH3wGLf3B69tQ7bLKoqh7JGg8pe\nb61AX8nbtwAMYus6DH5kvknWdtDhUvonaTyT5URwtX5a3KpA+Zf4vOcf+ZmA\nz+2tts2+Kn+iSPl7yNSD/Vt+bqqQB28lzmwT+rMB/QH9NNmjrq/o6h5h8p1H\nzw+QdwBSuQYrhM+f0JYD+x02RuuB8/8Aaf5bLmb/ALRdyxBcZaa7zEAffCuB\nn9etNmPhgcyeavI9KQ48l7knZM5C18rA5x7AMQcY9yR7nI1gw6Yf0RNT578o\ns5ZPI+4SD/kFzlLk/bihOP2OgVUP6IyfzR5trCFj8ibsijGehc5ouvzJcN/x\nOhFKuBhP5Z81U0Z9Ty7uhMfa81cpA/IFlH9TrKejChFHlTrBSDQBoA0B6GOQ\nayc9f7Vx/c6Az4P5hn7DQGcjkE4wQNAZtO3MAYxnQGfTsQcdY++gJCFgp6I9\nx76GVImKOYMODDI9sHsEfp7HQzy2ddRavTBqaJSU7LLkkr+n5aGdbMu21JIC\nNgj6H76Gg0W6pGBjvIxoBit8mQrKSCOtDKfQx2ypaMj659/01hrYjLRXnn/Z\nFLX28b7hiPGGD4C9CNAWajcgiYD6vCwVx9SA6j+Y5iaR0sW/XRqbt+2NZ626\n+ObywMdKjtSexSajlJHFT9Qjkgf+Boj9eoXFHTUkuxN2zLU7dvFTteSDMlFI\nQhyQHjJyhwBn2wOu+vvrEuWhWywRcJ3pyaquVMDuOBihH6lcuf0J/bUT5bJX\nP6LG8X3iGrsVVQQwmnFNUpI6gcA/NSMkE5P+z9yOsn2zrp4et9nC8jquWkON\nyp4qmyVUM0buienPxViuSJFXvHeMSHoavZCTXRQwpyjaLkdvaGNjBSRRczgZ\nABI+3+99frjUUa0dO2970fcaVSj0Hm9VVHzJTlQQPzPZGo7EkSJuSO+CKjjY\nO0EKcewPUaVyPc5wCf6YGomtEy2d81zoqvEcEbVbqcmOOMPj8iFDEH9ca0Yc\nToeS71DdQR0SDoBmBYZ+mByIP7D9tY5McYmO9OZGJlrnmI6wqg4P2y+f6YGN\nORjSMSoiWVMrSmYhiCjl3wAcZwcj9wP6axsjfI6uM0ZCPNSQDGeLuCR+iBgf\n7ftps03Ix6u60EPyfHzzsB7QwrEP0JYgn+mjkZUExTv27lpFb0KSGMsOjUT5\nJ/PAHf7E60czdVI88tTnHDQBoA0B6PXmwz0VXPNRyfExGRyMdOOznI+uPy17\nTyH6Wux1yh2eNwP1dj5b1YuJgw1BJCnII6wR3ryl2PZU9NHrKL68pbh2Z1PL\nwOGOQdV9aJ+vokoJAeOO/f8A5axpr0YT5EhC2FAx7nOgM6CQN17HrrQEnTOc\ngDrB0BMUNSMAE5H2P10NtnbPbsk1tIMAfzoPp17gfbQ1Mq31BAALdHvQDRbq\noFQAcYHf56AYLfVAMBnIOgJ6F4auCWirEWanqEMUsbdh0IIII+uQSNauJNVL\ngzTXzr4/r9j3RbjboJZZ9sOJYWAyau0y/LwOPdkB9I4JwVpiTnI1E4HSrs2i\nofI0MD1Vr3xbWM1PMqU8zp9Ucco2yCMg5xn7n9tRlhT2MFhnX0FYLTqJAByk\n+YjvHQBAyfzz76xxFmmPm3NxCwTyrFStI1QgV0eAoDgggg/L3kHGAeidWaZc\nSrbSrfYxDd71n+FU1BVG94VYBMg5HyqASQQD2D7fnq5Cwrqj4n0fFRdLhVEr\nBSuEx0JeMII+45dkf+XHv3rZ27MqsIkq6gpHU1sECjspApck/qTgD9Ez+eoJ\nS2Txi0T1FRWeNC0kLVLAe87lwPz4fy/1XWjaRYUTtW7JIDT0hAVchlhX5cfb\nAGB+51G5sNHTkpmaoqBDH/vPIqf37H/PWjYSF+67itNGGK1iyuOwYISwP/mb\nI/cawmbaICfclXVKfh6OtqifYlmCD9QMg/oNbBpGBU1V5x06QFvp8qftkk5/\npoa8UYM0VQQZKuYuAMssQdy35dDrWjYT0Rlh2Vdd+7ijtFtoJUVyDLMy8Vgi\nJ7YgnJ/IdZPX560fZhy0aIatnCDQBoA0B6MNfSK2aN2IQyMR0PqdfoCVcIe3\ns/M6z2+jrkqaSoch/p2GHR1y8zxOPnL/ACLR2vH+ey/HvdMjsRX5DDF1Azkn\nsa8R5T9K21pzo7R9J8P+r6MnUL+mZ1PIpI77Gfprx9tM6nwmtM9jCatXOD2i\nUp3xgg5GPpqIkJCDBBI0Bm0rgEA56OgJKmm4EAEe2gJmiquLBs5A7x99AZE9\nJkfE0Y/8Txg5x+Y/6aAyLfWqegT0QD1oBkoaoMAAdAMVFV4AJYYzj99YaZne

5. kitchen_23.png (similarity: 0.989)


"<img src='data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAMCAgICAgMCAgIDAwMDBAYEBAQE\nBAgGBgUGCQgKCgkICQkKDA8MCgsOCwkJDRENDg8QEBEQCgwSExIQEw8QEBD/\n2wBDAQMDAwQDBAgEBAgQCwkLEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQ\nEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBD/wAARCAEAAQADAREAAhEBAxEB/8QA\nHgAAAQQDAQEBAAAAAAAAAAAAAAMEBQYCBwgBCQr/xABXEAABAwMCAwUFBAQH\nCwcNAAABAgMRAAQFBiEHEjEIE0FRYSJxgZGhCRQV0SMyscEWGEJFldLwFyQl\nMzVSVWJlcuEmc3WDhbLTQ1NUY2R0gpKTlKKzw//EABoBAAEFAQAAAAAAAAAA\nAAAAAAABAgMEBQb/xAAtEQACAgEDAwMEAgIDAQAAAAAAAQIDEQQSIQUTMRUi\nQRQjMlFCUmFxJDOBsf/aAAwDAQACEQMRAD8A+qdIgChsDyRQIEigD2gU8M+F\nKB7QAUAFIsgFKAUAFABQAUAFABQB5J8qACT5UAe0AFAHkigTkBHhQHJ7SC4C\ngQKAweUoYCfX6UmQwE+v0oyGD8rFKKFABQAUAfocR2+Ozq4JTmswRMT+FPef\nuqNWJknbY4b7dfZ8c2Rmsv5/5Ke/Kn7kHbYqntw8AVRy5fLb/wCy3fyo3ITt\nTFR22eAxiMvltxP+THfypNyHdiZkO2pwKUARlsrv/sx2jchHSzIdtHgYf51y\nv9Gu0m9CdpmQ7Z3A0/ztlI/6Oco3C9uR7/HN4G/6Wyn9HOUbg7cjw9s3gft/\nhbKb/wCznKSViQqqZkrtlcEEK5HMplUqmIONdB+RpFamEqWjIdsfgidxlcr/\nAEc5+VO3Cdtnv8cbgjEnK5Qf9muflRuDtsP44/BH/SmU/o1z8qNwdtnh7Y/B\nEfzrlP6Oc/KjcHbYfxxeCR/nTKf0c5+VKmHbZkO2HwTP86ZQf9nOU4O2zw9s\nbgkP51yhHn+HOflQHbkefxx+CP8ApbKf0c5QHbZie2XwPT1yuU/o1ygTtMwP\nbQ4Fjb8Wyv8ARjv5UB22YK7anAdAJXmMqAOv+DHNvpSZBVyYmrtvcAkfrZjL\nAf8ARbv5Um5If2JsZ3Pbx7OdmkqutR5JpIEyvGugAeckVG7618j1pLX8Ec99\non2W2Gi8rWF+tA/WLWMedKT4AhIJE+FMeqrXyP8Aorf0IN/aNdmd+3VeWuV1\nHcW6J5nmsBclIjqT7NJ9XV+xfobhA/aT9lsJCzn85Hl+EOgj3gwfpR9XUH0N\nwgftNOymBP8ACLPGPLCPn91J9XAT6K39CKvtP+yWkSrUueAH+xH/AMqPrICf\nRWnwLq2VAoAKACgDuSxSe6E+KjHzIqmkaBLW0gjzJg0uAJNgQrbwoAfteG3h\nQAsgkgg+HSkwAqFx+sdqMAZh9IH64MUIA78eYNSpgN7wNXTRYeAKVQYJPgQQ\nfmBUdsd4sZbGSup9UZzWd7jb/UGUN2/h7FGPtT3DLXIwgnlSe7QnmICjuqT0\n8qjhU0x057kNkPqEE9Pf4VZIE2KpfSTIXMeE0ou4971P+f8AWgTIB8A+B+Jo\nDIt3/wDu/OhBvyYG6KCACAVGBKon0Hn8KkTF3Hvfnl3EnxApUDlIa3t4q2tn\n7oNOrDTal920nmWuATypHio9AJ3JAobBSkVrT+tzqG9ctGsLf2yWkFSnXkgI\nBBACSZkKMzHpUe8fuJtbhAgHelTyNya64m5TUGLdtnbHMOWNlcS04QEAB0bg\nBSgTJEwJ/kms/XWWVQ4NDQxjZLGDXwuV3rhVks9lbsmVA8zpQR6EAgbxuRWS\n7rZrlmqqYxWMGTSMcyskYtwkwQoqSoEj/eUD9KbvQ/toboz5s3Sizx5QpYnm\nbe5ZiOoCQOh/bvUkZP4EdQ1fyV2+t1a2mEKUST+jJUfeSdx7xTu4Rulfsr15\neZNRU4h1cA7DkSAAJkjb0p6mN7SXyRpcuHnHfvFw+pKQnYuKiSTOwI8ht6Uk\nrAdQ0fRahwKS2ggbE8h/eKarGN2Gr66U5IKACgAoA7psAe5lYggkfU1Uzg0C\n/aO4WcQtdYe+zmkNJZDMWmNcDVyqzQHVoXy8wAbB51SOnKDJ267UxyAhWYSe\nU7b07IDtt1I2O58hSgZ99PQEfEUABfgbk/MUAefeE9Qoj0B60AYquJ2IBilA\nx74TEb+8UZwJJ5ZmLrqFgn409SyOeEZouogbwT0mnCLAp3/lI/fQI8CiHYEG\nOtAzBmHCYhJE+NIGBVKlH2UpJPgOtJkEQOotGYzUOQt7y+uLppXKbZwNPFCX\nGgFKg7SJOxggEEiN5BvwSJFgZSwy2lDawlKBABUVbRtvvPzpe6I65GRWyASV\nLJ6+ykQfmaa7QUJDVLjQcf8A73MlYPNzxJ5UiSI8gB18Ki7g/tmanklJCWUA\nkdYM/UkUquwL2yk8TEd5hbdBgReNK2AE+yvy99UNdZ9vk0OmRkrcGpHrh4Ol\nLYSAj2d0k7AnyO9Y8pZ8G84tywN0P3RdjvAANgAkevnNImvgTGCYxuEtbooN\nxcBADa+sD2toHoetSxcvgjckVvUDlyy8W27x1SAJA5jA6bx0pylgikm/gr5a\nWt0LKzzEgQZ39/nUimhm1/o8+7ybhYR+oEKO8CCSJ8+pHp5ijI18jN5grc5R\nuCehpjeA2M1RXWHGhQAUAFAHdNgsFqCBPOodfWqmDQJJh3kkJcKJBBjxnrNN\ncQHKHiVCVEgedPYCgeUkeyYFIAF9Z6kmOk0jYuQLx6KI93Sm7hMZMVXCAY3N\nG5gYm4Co5Qduu9LvFSMw4FbgCffS7kP2CqVRsYAPmYFG5De2LIISQVvNpB6S\nufoJIpO7gO2ZJuWgSJWryIG31IP0o76Q7YKJuxPstGentGYqKWoY5VCiX3ok\nEDygA01Wtg6hWXzH6QkeQMbe6h2MEjEt/p2JgQsn5oVSbmOHaUkSBvT0BmG+\nhJ3G8CnAJhEuOnr7QMR/qigD0pgH2fpQBUOIVuXcRbpkSbxBPmAELJqhrl9s\nv9PebDU7jCRcvcogBahuOmxrGOgURJDHKsiBuonz3oYNGSFOM23MPaHfrMqk\nwCdh1jp5+XhV6K+2Z8l9wg7xsOqBWJUYHkJ/sKgyTRjjyRq0DYJMAGdx13/4\n05MSUcmJEN3qu5EBlAn/AKwD99SJ4RDZHDGikFSyQNx5eG9M3bh0eEaarrjh\nwoAKACgDtmyv2uUysD21dPeapKRoD5GRYQCCofE07IB+OMJBUHQfhUbkOism\nC8+gAkKAA3JOwpNxJsMDqJoiEvJn03/ZTJWoNgJzyVKg86vOEkftpndDaKpy\ni1GQyQB0JVuR7opHaGwWReXCvFCZ8k/smaa7R8YDll59SpDqgSOkwPkKZuZJ\nhC7LZ/WUSDOxFG5hgeNtExtPifzpNzYYFkMpiJiKOWGBwhuSOUeMdaVgkOUI\nggCJ6VJDANDhLZTIIjyp5EYLRyuNEx+sf+6qnpAOm0yB5mlQCvKnl6bj1pQE\n2UpK3TG8j9goAUUgEbCgCra7ZUrFNgdQ+OkbDkVVDXP7ZodNX3DVVyylN06Q\nBHeKn5n8xWJk6PwIpQnZW4ggn13FNyNY3dZK2xEgBxYIjYmVEb+gH1rSh/1m\nc39whbxmFyAdok+vjVZlmTwR7tuG/DmI2I9N6ciPOTxhsC2vl8xIDYTBiSAo\nGSOnn86lXKIpvkYBHtEiNyJj31E/ax6WTSVdicIFABQAUAbxw+ucndOu27zI\nSoAuBSXDB3G0Ebddt/CsjcbbrwTrGbyb4kuJECNgTTXIFEdsPXrigXLlZneA

[["house_data_png/bed_1009.png", "house_data_png/bed_1253.png", "house_data_png/din_526.png", "house_data_png/din_1260.png", "house_data_png/kitchen_23.png"], [0.9927145838737488, 0.9927145838737488, 0.9897346496582031, 0.9895848631858826, 0.989266037940979]]